install **libraries**

In [ ]:
#Hagar Mahmoud Mohamed      420220824
#Rama Mohamed Mutaz         420220953
#Toqa Abselnasser           420220640
#Rowaida Gamal Hassan       420221020

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

download data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment140


In [ ]:
import pandas as pd
import zipfile
import os


print("Path to dataset files:", path)


zip_path = path
extract_dir = "/kaggle/input/sentiment140_extracted"


if zip_path.endswith(".zip") and not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    data_dir = extract_dir
else:
    data_dir = path

csv_file = None
for root, dirs, files in os.walk(data_dir):
    for f in files:
        if f.lower().endswith(".csv"):
            csv_file = os.path.join(root, f)
            break
    if csv_file:
        break

if csv_file is None:
    raise FileNotFoundError("لم يتم العثور على ملف CSV داخل المسار الناتج.")

print("Reading file:", csv_file)

df = pd.read_csv(csv_file, encoding='latin-1', header=None)

df = df[[0, 5]]
df.columns = ['polarity', 'text']

print(df.head())
print(df.tail())

Path to dataset files: /kaggle/input/sentiment140
Reading file: /kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv
   polarity                                               text
0         0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1         0  is upset that he can't update his Facebook by ...
2         0  @Kenichan I dived many times for the ball. Man...
3         0    my whole body feels itchy and like its on fire 
4         0  @nationwideclass no, it's not behaving at all....
         polarity                                               text
1599995         4  Just woke up. Having no school is the best fee...
1599996         4  TheWDB.com - Very cool to hear old Walt interv...
1599997         4  Are you ready for your MoJo Makeover? Ask me f...
1599998         4  Happy 38th Birthday to my boo of alll time!!! ...
1599999         4  happy #charitytuesday @theNSPCC @SparksCharity...


Keep Only Positive and Negative Sentiments

In [ ]:
df = df[df.polarity != 2]

df['polarity'] = df['polarity'].map({0: 0, 4: 1})

print(df['polarity'].value_counts())

polarity
0    800000
1    800000
Name: count, dtype: int64


Step 4: Clean the Tweets

In [ ]:
def clean_text(text):
    return text.lower()

df['clean_text'] = df['text'].apply(clean_text)

print(df[['text', 'clean_text']].head())

                                                text  \
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1  is upset that he can't update his Facebook by ...   
2  @Kenichan I dived many times for the ball. Man...   
3    my whole body feels itchy and like its on fire    
4  @nationwideclass no, it's not behaving at all....   

                                          clean_text  
0  @switchfoot http://twitpic.com/2y1zl - awww, t...  
1  is upset that he can't update his facebook by ...  
2  @kenichan i dived many times for the ball. man...  
3    my whole body feels itchy and like its on fire   
4  @nationwideclass no, it's not behaving at all....  


Step 5: Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'],
    df['polarity'],
    test_size=0.2,
    random_state=42
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))

Train size: 1280000
Test size: 320000


Step 6: Perform Vectorization

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("TF-IDF shape (train):", X_train_tfidf.shape)
print("TF-IDF shape (test):", X_test_tfidf.shape)

TF-IDF shape (train): (1280000, 5000)
TF-IDF shape (test): (320000, 5000)


Step 7: Train Bernoulli Naive Bayes model

In [ ]:
bnb = BernoulliNB()
bnb.fit(X_train_tfidf, y_train)

bnb_pred = bnb.predict(X_test_tfidf)

print("Bernoulli Naive Bayes Accuracy:", accuracy_score(y_test, bnb_pred))
print("\nBernoulliNB Classification Report:\n", classification_report(y_test, bnb_pred))

Bernoulli Naive Bayes Accuracy: 0.766478125

BernoulliNB Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.75      0.76    159494
           1       0.76      0.78      0.77    160506

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



Step 9: Train Support Vector Machine (SVM) model

In [ ]:
svm = LinearSVC(max_iter=1000)
svm.fit(X_train_tfidf, y_train)

svm_pred = svm.predict(X_test_tfidf)

print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("\nSVM Classification Report:\n", classification_report(y_test, svm_pred))

SVM Accuracy: 0.79528125

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.78      0.79    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000



Step 10: Train Logistic Regression model

In [ ]:
logreg = LogisticRegression(max_iter=100)
logreg.fit(X_train_tfidf, y_train)

logreg_pred = logreg.predict(X_test_tfidf)

print("Logistic Regression Accuracy:", accuracy_score(y_test, logreg_pred))
print("\nLogistic Regression Classification Report:\n", classification_report(y_test, logreg_pred))

Logistic Regression Accuracy: 0.79539375

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.78      0.79    159494
           1       0.79      0.81      0.80    160506

    accuracy                           0.80    320000
   macro avg       0.80      0.80      0.80    320000
weighted avg       0.80      0.80      0.80    320000



Step 11: Make Predictions on sample Tweets

In [ ]:
sample_tweets = ["I love this!", "I hate that", "It was okay, not great."]
sample_vec = vectorizer.transform(sample_tweets)

print("\nSample Predictions:")
print("BernoulliNB:", bnb.predict(sample_vec))
print("SVM:", svm.predict(sample_vec))
print("Logistic Regression:", logreg.predict(sample_vec))


Sample Predictions:
BernoulliNB: [1 0 1]
SVM: [1 0 1]
Logistic Regression: [1 0 1]
